In [3]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split

import os
import shutil
import time
import matplotlib.pyplot as plt
import math
import pickle

from dataset import prepare_dataset
from dataset import ToFloatTensorInZeroOne
from model import LSTM_with_EFFICIENTNET

In [6]:
colab=False
batch_size=32
done_eppoch=0
train_epochs=50

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if colab:
    root = '/content/drive/MyDrive'
else:
    root = './'

location = {
        'video_path': os.path.join(root, '../datasets/hmdb51dataset/video'),
        'annotation_path': os.path.join(root, '../datasets/hmdb51dataset/annotation'),
    }

In [17]:
# Preprocessing dataset
transform_train = transforms.Compose([
    ToFloatTensorInZeroOne(),
    transforms.Resize([256,342]),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomCrop(224)
])
transform_test = transforms.Compose([
    ToFloatTensorInZeroOne(),
    transforms.Resize([256,342]),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.CenterCrop(224)
])

dataset_train_val = torchvision.datasets.HMDB51(
    root=location['video_path'],
    annotation_path=location['annotation_path'],
    frames_per_clip=10,
    step_between_clips=1,
    train=True,
    transform=transform_train
)
dataset_test = torchvision.datasets.HMDB51(
    root=location['video_path'],
    annotation_path=location['annotation_path'],
    frames_per_clip=10,
    step_between_clips=1,
    train=False,
    transform=transform_test
)

  0%|          | 0/423 [00:00<?, ?it/s]

  0%|          | 0/423 [00:00<?, ?it/s]

In [18]:

dataset_train_val.metadata

{'video_paths': ['./../datasets/hmdb51dataset/video/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_0.avi',
  './../datasets/hmdb51dataset/video/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_1.avi',
  './../datasets/hmdb51dataset/video/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_2.avi',
  './../datasets/hmdb51dataset/video/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np1_ba_goo_4.avi',
  './../datasets/hmdb51dataset/video/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np1_ri_med_3.avi',
  './../datasets/hmdb51dataset/video/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np2_le_goo_0.avi',
  './../datasets/hmdb51dataset/video/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np2_le_goo_1.avi',
  './../datasets/hmdb51dataset/video/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np2_le_med_2.avi',
  './../datasets/hmdb51dataset/video/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_ba_med_3.avi',
  './../dat

In [16]:
print((dataset_train_val.metadata['video_paths'][10]), len(dataset_train_val.metadata['video_pts'][10]))

./../datasets/hmdb51dataset/video/brush_hair/Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_2.avi 647


In [11]:
loader_train=DataLoader(dataset_train_val, batch_size=batch_size, shuffle=True, drop_last=False)
loader_test=DataLoader(dataset_test, batch_size=batch_size, shuffle=False, drop_last=False)

In [12]:
dataset_len = len(dataset_train_val)
test_len = len(dataset_test)

In [13]:
print(dataset_len, test_len)

63834 26470
